In [17]:
from gensim.models import Word2Vec
import helper
import pandas as pd
import csv 
from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics

In [18]:
# ### Handle input

# trainingdata = pd.read_csv("../datasets/train/SemEval2018-T3-train-taskA_emoji.txt", delimiter = "\t",  quoting=csv.QUOTE_NONE, header=0)
# tweets = trainingdata['Tweet text']

In [19]:
# define an empty list
tweets = []

# open file and read the content in a list
with open('tweets_corpus.txt', 'r') as filehandle:  
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        tweets.append(currentPlace)
        


In [20]:
corpus = helper.allbags(tweets)

In [21]:
### Word2Vec maker

model = Word2Vec(corpus, min_count=5, size=200, window=8)
model.save("word2vec.model")

In [22]:
words = model.wv.index2word
X = model.wv[words]

In [23]:
model.wv.most_similar(positive=['stupid'], negative=[])

[('realise', 0.9363247156143188),
 ('offend', 0.9210367202758789),
 ('dumb', 0.9173310399055481),
 ('pose', 0.9084862470626831),
 ('fucking', 0.9083322286605835),
 ('soo', 0.9072073698043823),
 ('dislike', 0.9044171571731567),
 ('subtweet', 0.9040661454200745),
 ('hashtags', 0.9028514623641968),
 ('rant', 0.8909350633621216)]

In [24]:
### Gebruik sklearn want multithread

kmeans = cluster.KMeans(n_clusters=2000, precompute_distances=True)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")
print (silhouette_score)

Cluster id labels for inputted data
[ 358   73  289 ...  126 1043 1477]
Centroids data
[[-7.4900389e-02  2.9272042e-02 -4.5278311e-02 ... -1.4119858e-01
  -1.9072019e-02 -9.5745929e-02]
 [-3.2571342e-02  6.4774282e-02  5.9397500e-02 ... -7.3749140e-02
  -6.9815598e-02 -2.0614229e-02]
 [ 4.4399610e-01  6.6564620e-01  5.0117433e-01 ... -4.2803681e-01
  -3.2921427e-01 -3.3334041e-01]
 ...
 [-6.8395332e-02  1.1775652e-01  3.3680033e-02 ... -2.1099424e-01
  -6.7458190e-02 -9.5007695e-02]
 [ 4.8962790e-03  5.3798854e-02  7.7379838e-02 ... -1.9991070e-01
  -1.6497010e-01 -1.4336881e-01]
 [ 2.7769245e-04  4.1168354e-02  2.6051547e-02 ... -1.5872923e-01
  -1.5470338e-01 -8.8443935e-02]]
Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):
-572.60626
Silhouette_score: 
0.021034608


In [25]:
len(centroids)

2000

In [26]:
kmeans.predict([model.wv['dog'],model['haha']])

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([671, 278], dtype=int32)

In [11]:
inputbag = helper.allbags(["Hi my name is jeff", "I think you're cool"])
resulting_clusters = []

for row in inputbag:
    clusters_row = []
    filtered_words = list(filter(lambda x: x in words, row))
    clusters_row.append(list(kmeans.predict(model[filtered_words])))
    resulting_clusters.append(clusters_row)
            
resulting_clusters

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


[[[1845, 57, 432, 78, 944]], [[63, 126, 232, 1498]]]

In [12]:
filtered_words

['i', 'think', "you're", 'cool']

In [27]:
from sklearn.externals import joblib

## Save the classifier
_ = joblib.dump(kmeans, "kmeansclass.model")

In [14]:
### Load the classifier
kmeansmodel = joblib.load("kmeansclass.model")
w2vmodel = Word2Vec.load("word2vec.model")
vocab = w2vmodel.wv.index2word

inputbag = helper.allbags(["Hi my name is jeff", "I think you're cool"])
resulting_clusters = []

for row in inputbag:
    filtered_words = list(filter(lambda x: x in vocab, row))
    resulting_clusters.append(list(kmeansmodel.predict(model[filtered_words])))
            
resulting_clusters

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


[[152, 1702, 1658, 1658, 1350], [1719, 1658, 1719, 1296]]

In [20]:
import numpy as np
onehotvec = np.zeros((len(resulting_clusters), 2000))
for i in range(len(resulting_clusters)):
    onehotvec[i, resulting_clusters[i]] = 1


In [23]:
np.where(onehotvec)

(array([0, 0, 0, 0, 1, 1, 1]),
 array([ 152, 1350, 1658, 1702, 1296, 1658, 1719]))